The task is to train a network to discriminate between sonar signals bounced off a metal cylinder and those bounced off a roughly cylindrical rock.

In [90]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline

In [91]:
import warnings 
warnings.filterwarnings(action='ignore')

# Step-1: Data Gathering

In [92]:
df=pd.read_csv('sonar_dataset.csv',header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,Rock
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,Rock
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,Rock
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,Rock
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,Rock


In [93]:
df[0:1]

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.02,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.018,0.0084,0.009,0.0032,Rock


In [94]:
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
196,0.0050,0.0017,0.0270,0.0450,0.0958,0.0830,0.0879,0.1220,0.1977,0.2282,...,0.0165,0.0056,0.0010,0.0027,0.0062,0.0024,0.0063,0.0017,0.0028,Mine
78,0.0231,0.0351,0.0030,0.0304,0.0339,0.0860,0.1738,0.1351,0.1063,0.0347,...,0.0106,0.0097,0.0022,0.0052,0.0072,0.0056,0.0038,0.0043,0.0030,Rock
114,0.0114,0.0222,0.0269,0.0384,0.1217,0.2062,0.1489,0.0929,0.1350,0.1799,...,0.0269,0.0152,0.0257,0.0097,0.0041,0.0050,0.0145,0.0103,0.0025,Mine
183,0.0096,0.0404,0.0682,0.0688,0.0887,0.0932,0.0955,0.2140,0.2546,0.2952,...,0.0237,0.0078,0.0144,0.0170,0.0012,0.0109,0.0036,0.0043,0.0018,Mine
175,0.0294,0.0123,0.0117,0.0113,0.0497,0.0998,0.1326,0.1117,0.2984,0.3473,...,0.0056,0.0104,0.0079,0.0014,0.0054,0.0015,0.0006,0.0081,0.0043,Mine


In [95]:
df.shape

(208, 61)

In [96]:
df.isna().sum() # isnull() and isna() literally does the same things. isnull() is just an alias of the isna() method as shown in pandas source code.

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [97]:
df.columns # 60 is the target column

Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
            17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
            34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
            51, 52, 53, 54, 55, 56, 57, 58, 59, 60],
           dtype='int64')

In [98]:
df[60]

0      Rock
1      Rock
2      Rock
3      Rock
4      Rock
       ... 
203    Mine
204    Mine
205    Mine
206    Mine
207    Mine
Name: 60, Length: 208, dtype: object

In [99]:
df[60].value_counts()

Mine    111
Rock     97
Name: 60, dtype: int64

# Step-2: Data Preprocessing

In [100]:
x=df.drop(60, axis=1) # axis=1/column refrers column 
x.shape

(208, 60)

In [101]:
y=df[60]
y.shape

(208,)

    | One Hot Encoding

In [102]:
y=pd.get_dummies(y,drop_first=True)
y # 0--> Mine and 1--> Rock 

,Rock
0,1
1,1
2,1
3,1
4,1
...,...
203,0
204,0
205,0
206,0


In [103]:
y.value_counts()

Rock
0       111
1        97
dtype: int64

    | Train and Test split

In [104]:
from sklearn.model_selection import train_test_split

In [105]:
xtrain, xtest, ytrain, ytest=train_test_split(x,y,test_size=0.25, random_state=5)

In [106]:
xtrain.shape,xtest.shape

((156, 60), (52, 60))

In [107]:
xtrain

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
50,0.0353,0.0713,0.0326,0.0272,0.0370,0.0792,0.1083,0.0687,0.0298,0.0880,...,0.0098,0.0163,0.0242,0.0043,0.0202,0.0108,0.0037,0.0096,0.0093,0.0053
55,0.0201,0.0116,0.0123,0.0245,0.0547,0.0208,0.0891,0.0836,0.1335,0.1199,...,0.0032,0.0076,0.0045,0.0056,0.0075,0.0037,0.0045,0.0029,0.0008,0.0018
77,0.0336,0.0294,0.0476,0.0539,0.0794,0.0804,0.1136,0.1228,0.1235,0.0842,...,0.0033,0.0150,0.0111,0.0032,0.0035,0.0169,0.0137,0.0015,0.0069,0.0051
23,0.0115,0.0150,0.0136,0.0076,0.0211,0.1058,0.1023,0.0440,0.0931,0.0734,...,0.0107,0.0091,0.0016,0.0084,0.0064,0.0026,0.0029,0.0037,0.0070,0.0041
45,0.0408,0.0653,0.0397,0.0604,0.0496,0.1817,0.1178,0.1024,0.0583,0.2176,...,0.0066,0.0062,0.0129,0.0184,0.0069,0.0198,0.0199,0.0102,0.0070,0.0055
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144,0.0299,0.0688,0.0992,0.1021,0.0800,0.0629,0.0130,0.0813,0.1761,0.0998,...,0.0342,0.0459,0.0277,0.0172,0.0087,0.0046,0.0203,0.0130,0.0115,0.0015
118,0.0363,0.0478,0.0298,0.0210,0.1409,0.1916,0.1349,0.1613,0.1703,0.1444,...,0.0250,0.0115,0.0190,0.0055,0.0096,0.0050,0.0066,0.0114,0.0073,0.0033
189,0.0158,0.0239,0.0150,0.0494,0.0988,0.1425,0.1463,0.1219,0.1697,0.1923,...,0.0223,0.0121,0.0108,0.0057,0.0028,0.0079,0.0034,0.0046,0.0022,0.0021
206,0.0303,0.0353,0.0490,0.0608,0.0167,0.1354,0.1465,0.1123,0.1945,0.2354,...,0.0042,0.0086,0.0046,0.0126,0.0036,0.0035,0.0034,0.0079,0.0036,0.0048


In [108]:
ytrain

,Rock
50,1
55,1
77,1
23,1
45,1
...,...
144,0
118,0
189,0
206,0


# Step-3: Build an ANN model

In [109]:
model=keras.Sequential([
    keras.layers.Dense(60, input_dim=60,activation='relu'),
    keras.layers.Dense(30 ,activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])


In [111]:
model.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy']) # compile the model
model .fit(xtrain,ytrain, epochs=100,batch_size=8) # train the model ANN | batch_size=8 refers Batch GD

Epoch 1/100
20/20 [==============================] - 2s 4ms/step - loss: 0.6579 - accuracy: 0.7564
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6424 - accuracy: 0.6218
Epoch 3/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6108 - accuracy: 0.7949
Epoch 4/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5758 - accuracy: 0.7821
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5428 - accuracy: 0.7821
Epoch 6/100
20/20 [==============================] - 0s 5ms/step - loss: 0.5076 - accuracy: 0.7564
Epoch 7/100
20/20 [==============================] - 0s 4ms/step - loss: 0.4800 - accuracy: 0.7756
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4743 - accuracy: 0.7821
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4424 - accuracy: 0.8013
Epoch 10/100
20/20 [==============================] - 0s 5ms/step - loss: 0.4124 - accuracy: 0.8205
Epoch 11/

In [112]:
model.evaluate(xtest,ytest) # model performence/accuracy

2/2 [==============================] - 0s 4ms/step - loss: 0.6168 - accuracy: 0.8846


[0.6168067455291748, 0.8846153616905212]

In [125]:
y_pred=model.predict(xtest).reshape(-1)
print(y_pred[:10])

y_pred=np.round(y_pred) # round the values to nearest integer i.e. 0 or 1
print(y_pred[:10])


2/2 [==============================] - 0s 8ms/step
[1.8927301e-06 9.9550945e-01 4.6162963e-02 7.7074687e-03 9.9831897e-01
 1.9377205e-06 9.9954325e-01 9.9999887e-01 4.0376824e-05 9.9998909e-01]
[0. 1. 0. 0. 1. 0. 1. 1. 0. 1.]


In [126]:
ytest[:10]

,Rock
156,0
21,1
193,0
159,0
34,1
196,0
6,1
25,1
115,0
51,1


In [128]:
from sklearn.metrics import confusion_matrix, classification_report

In [130]:
print(classification_report(ytest,y_pred))  # classification

              precision    recall  f1-score   support

           0       0.93      0.88      0.90        32
           1       0.82      0.90      0.86        20

    accuracy                           0.88        52
   macro avg       0.88      0.89      0.88        52
weighted avg       0.89      0.88      0.89        52



    | Confusion metrics

# Steps-5: Dropout Regularization

In [135]:
model_d=keras.Sequential([
    keras.layers.Dense(60, input_dim=60,activation='relu'),
    keras.layers.Dropout(.5), # it will drop 50% of the neurons | reduce accuracy
    keras.layers.Dense(30 ,activation='relu'),
    keras.layers.Dropout(.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(.5),
    keras.layers.Dense(1, activation='sigmoid')
])


In [136]:
model_d.compile(optimizer='adam',loss='binary_crossentropy', metrics=['accuracy']) # compile the model
model_d.fit(xtrain,ytrain, epochs=100,batch_size=8) # train the model ANN | batch_size=8 refers Batch GD

Epoch 1/100
20/20 [==============================] - 2s 4ms/step - loss: 0.7811 - accuracy: 0.5128
Epoch 2/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7503 - accuracy: 0.4487
Epoch 3/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7042 - accuracy: 0.5256
Epoch 4/100
20/20 [==============================] - 0s 6ms/step - loss: 0.7182 - accuracy: 0.4295
Epoch 5/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7074 - accuracy: 0.5128
Epoch 6/100
20/20 [==============================] - 0s 7ms/step - loss: 0.6927 - accuracy: 0.5769
Epoch 7/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6984 - accuracy: 0.5064
Epoch 8/100
20/20 [==============================] - 0s 5ms/step - loss: 0.6733 - accuracy: 0.5321
Epoch 9/100
20/20 [==============================] - 0s 5ms/step - loss: 0.7039 - accuracy: 0.5128
Epoch 10/100
20/20 [==============================] - 0s 6ms/step - loss: 0.6997 - accuracy: 0.5064
Epoch 11/

In [137]:
model_d.evaluate(xtest,ytest) # model performence/accuracy

2/2 [==============================] - 1s 10ms/step - loss: 0.3652 - accuracy: 0.8269


[0.36517736315727234, 0.8269230723381042]

    | Prediction

In [138]:
y_pred=model_d.predict(xtest).reshape(-1) 
print(y_pred[:10])

y_pred=np.round(y_pred) # round the values to nearest integer i.e. 0 or 1
print(y_pred[:10])

2/2 [==============================] - 0s 4ms/step
[0.07044723 0.862336   0.1916399  0.31626126 0.9440841  0.04923764
 0.89104736 0.9993277  0.22913934 0.99992317]
[0. 1. 0. 0. 1. 0. 1. 1. 0. 1.]


In [139]:
ytest[:10]

,Rock
156,0
21,1
193,0
159,0
34,1
196,0
6,1
25,1
115,0
51,1
